In [11]:
from numba import njit
import numpy as np

@njit(fastmath=True)
def montecarlo_pi(n):
    count = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if x**2 + y**2 < 1:
            count += 1
    return 4 * count / n    

print(montecarlo_pi(100000000))

3.14158708
